In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
from sklearn.metrics import accuracy_score,classification_report, f1_score

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# === 1. 載入 .tflite 模型 ===
interpreter = tf.lite.Interpreter(model_path="test.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_index = input_details[0]['index']
output_index = output_details[0]['index']

# 輸入尺寸
_, height, width, channels = input_details[0]['shape']

In [4]:
# === 2. 圖片預處理函式 ===

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (width, height))
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)  # 加入 batch 維度
    return img

In [5]:
# === 3. 讀取測試圖片與標籤 ===
def load_test_data(img_dir):
    X = []
    y = []
    class_names = sorted(os.listdir(img_dir))  # 每個子資料夾為一個類別
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}

    for class_name in class_names:
        class_folder = os.path.join(img_dir, class_name)
        if not os.path.isdir(class_folder):
            continue
        for fname in os.listdir(class_folder):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(class_folder, fname)
                img = preprocess_image(img_path)
                if img is not None:
                    X.append(img)
                    y.append(class_to_idx[class_name])
    return X, y, class_names

# 設定測試資料資料夾
test_dir = "trashbox"  # ← 替換為實際測試資料夾路徑
X, y_true, class_names = load_test_data(test_dir)

In [6]:
# === 4. 執行推論 ===
y_pred = []
for img in X:
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    pred = np.argmax(output, axis=-1)[0]
    y_pred.append(pred)

In [7]:
# === 5. 計算 F1-score 與分類報告 ===
accuracy = accuracy_score(y_true, y_pred)
print(f"準確率: {accuracy}")

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

準確率: 0.7019250460782306
              precision    recall  f1-score   support

          BG     0.7051    1.0000    0.8271       373
          PG     0.7788    0.5771    0.6630      3734
       metal     0.4634    0.6353    0.5359      1593
       paper     0.7734    0.8153    0.7938      4066

    accuracy                         0.7019      9766
   macro avg     0.6802    0.7569    0.7049      9766
weighted avg     0.7223    0.7019    0.7030      9766

